In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# iterate through express:
# then iterate through the vad
# then iterate through the dct 2-15
# plot dendro, save
# iterate through cut 1-9
# plot each cluster, save

from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

def series_to_df(series):
  df = pd.DataFrame(series)
  df = df.rename(columns={0: emo_name})
  df['s_index'] = df.index

  return df

In [4]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

paths = ["./drive/My Drive/Colab Notebooks/Analysis/guten/sentences_express_predictions", "./drive/My Drive/Colab Notebooks/Analysis/guten/sentences_react_predictions"]
save_paths = ["./drive/My Drive/Colab Notebooks/Analysis/guten/sentences_express_images", "./drive/My Drive/Colab Notebooks/Analysis/guten/sentences_react_images"]
all_trans = ['dct']
low_pass_sizes = list(range(2,9))
low_pass_sizes.reverse()
# dim_names = ['V','A','D']
dim_names = []
# dim_labels = ['Valence Score','Arousal Score','Dominance Score']
dim_labels = []
cat_names = ['haha_count','wow_count','angry_count','sad_count']
cat_labels = ['Haha Ratio','Wow Ratio','Angry Ratio','Sad Ratio']
cuts = list(range(1,10))

for i, path in enumerate(paths):
  print('path:', path)
  for trans in all_trans:
    print('trans:', trans)
    if i == 0:
      emo_names = dim_names
      emo_labels =  dim_labels
    else:
      emo_names = cat_names
      emo_labels =  cat_labels
    
    for emo_index, emo_name in enumerate(emo_names):
      print('emo:',emo_name)
      for low_pass in low_pass_sizes:
        print('lowpass:',low_pass)
        filename = str(low_pass) + '.npy'
        load_path = path + '/' + trans+ '/' + emo_name + '/' + filename
        X = np.load(load_path, allow_pickle=True)
        X = [x for x in X if np.any(np.isfinite(x))]
        X = np.array(X)

        model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
        model = model.fit(X)
        
        fig = plt.figure()
        plt.title('Hierarchical Clustering Dendrogram')
        # plot the top three levels of the dendrogram
        # plot_dendrogram(clustering, truncate_mode=None, p=3)
        plot_dendrogram(model, truncate_mode='level', p=3)
        plt.xlabel("Number of points in node (or index of point if no parenthesis).")

        save_filename = 'low_%s_den.png' % low_pass
        save_path = save_paths[i] + '/' + trans+ '/' + emo_name + '/' + save_filename
        plt.savefig(save_path)
        plt.close(fig)
  
        for cut in cuts:
          print('cut:',cut)
          clustering = AgglomerativeClustering(n_clusters=cut).fit(X)
          labels = clustering.labels_
          cluster_ids = set(labels)

          for c_id in cluster_ids:
            print('c id:',c_id, cluster_ids)
            fig = plt.figure()
            matches = labels == c_id
            matching_series = X[matches]

            all_df = series_to_df(matching_series[0])
            for series in matching_series[1:]:
              df = series_to_df(series)
              ax = sns.lineplot(x="s_index", y=emo_name, data=df, alpha= 0.2, color= 'lightblue')
              
              
              all_df = pd.concat([all_df, df])
            # ax = sns.regplot(x="s_index", y=emo_name, data=all_df, lowess=False, scatter=False)
            # ax = sns.lineplot(x="s_index", y=emo_name, data=all_df, alpha= 1,color= 'red')
            ax = sns.lineplot(x="s_index", y=emo_name, data=all_df, alpha= 1, ci='sd',color= 'red')
            x_label = 'Story Progress (%)'
            y_label = emo_labels[emo_index]
            ax.set_title('%s vs. %s' % (y_label, x_label))
            ax.set_xlabel(x_label)
            ax.set_ylabel(y_label)

            fig.set_size_inches(8,4)
            save_filename = 'low_%s_cut_%s_cluster_%s.png' % (low_pass, cut, c_id)
            save_path = save_paths[i] + '/' + trans+ '/' + emo_name + '/' + save_filename
            plt.savefig(save_path)
            plt.close(fig)

          save_filename = 'low_%s_cut_%s_labels.npy' % (low_pass, cut)
          save_path = save_paths[i] + '/' + trans+ '/' + emo_name + '/' + save_filename
          np.save(save_path, labels, allow_pickle=True)
            

path: ./drive/My Drive/Colab Notebooks/Analysis/guten/sentences_express_predictions
trans: dct
path: ./drive/My Drive/Colab Notebooks/Analysis/guten/sentences_react_predictions
trans: dct
emo: haha_count
lowpass: 8
cut: 1
c id: 0 {0}
cut: 2
c id: 0 {0, 1}
c id: 1 {0, 1}
cut: 3
c id: 0 {0, 1, 2}
c id: 1 {0, 1, 2}
c id: 2 {0, 1, 2}
cut: 4
c id: 0 {0, 1, 2, 3}
c id: 1 {0, 1, 2, 3}
c id: 2 {0, 1, 2, 3}
c id: 3 {0, 1, 2, 3}
cut: 5
c id: 0 {0, 1, 2, 3, 4}
c id: 1 {0, 1, 2, 3, 4}
c id: 2 {0, 1, 2, 3, 4}
c id: 3 {0, 1, 2, 3, 4}
c id: 4 {0, 1, 2, 3, 4}
cut: 6
c id: 0 {0, 1, 2, 3, 4, 5}
c id: 1 {0, 1, 2, 3, 4, 5}
c id: 2 {0, 1, 2, 3, 4, 5}
c id: 3 {0, 1, 2, 3, 4, 5}
c id: 4 {0, 1, 2, 3, 4, 5}
c id: 5 {0, 1, 2, 3, 4, 5}
cut: 7
c id: 0 {0, 1, 2, 3, 4, 5, 6}
c id: 1 {0, 1, 2, 3, 4, 5, 6}
c id: 2 {0, 1, 2, 3, 4, 5, 6}
c id: 3 {0, 1, 2, 3, 4, 5, 6}
c id: 4 {0, 1, 2, 3, 4, 5, 6}
c id: 5 {0, 1, 2, 3, 4, 5, 6}
c id: 6 {0, 1, 2, 3, 4, 5, 6}
cut: 8
c id: 0 {0, 1, 2, 3, 4, 5, 6, 7}
c id: 1 {0, 1, 2, 3, 4, 